In [2]:
import cPickle as pickle
import pandas as pd
from similarity_utils import filter_df_by_sim_among_positives 

def filter_df_by_seq_len(df, lengths=(20, 700)):
    min_length, max_length = lengths
    return df[(df['sequence'].apply(len) >= min_length) & (df['sequence'].apply(len) <= max_length)].reset_index(drop=True)

# Uniprot

In [2]:
from utils import *

In [ ]:
with open('pickles/go_dict.pkl', 'r') as fin:
    go_dict = pickle.load(fin)
with open('pickles/protein_dict.pkl', 'r') as fin:
    protein_dict = pickle.load(fin)
with open('pickles/gene_dict.pkl', 'r') as fin:
    gene_dict = pickle.load(fin)

In [6]:
columns = protein_dict.itervalues().next().keys()
columns_data = [[] for c in columns]
allergen_set = set()
for k,v in protein_dict.iteritems():
    if v['allergen'] == True and v['sequence'] not in allergen_set:
        allergen_set.add(v['sequence'])
        for i,c in enumerate(columns):
            columns_data[i].append(v[c])
uniprot_allergen_df = pd.DataFrame(dict(tuple(zip(columns, columns_data))))

columns = protein_dict.itervalues().next().keys()
columns_data = [[] for c in columns]
non_allergen_set = set()
for k,v in protein_dict.iteritems():
    if v['sequence'] not in allergen_set and v['sequence'] not in non_allergen_set:
        non_allergen_set.add(v['sequence'])
        for i,c in enumerate(columns):
            columns_data[i].append(v[c])
uniprot_non_allergen_df = pd.DataFrame(dict(tuple(zip(columns, columns_data))))

In [7]:
uniprot_allergen_df.head(10)

,acc_num,allergen,db,ec_num,entry_name,gene_names,organism_name,other_acc_nums,protein_name,sequence
0,Q4WCS7,True,Swiss-Prot,1.11.1.15,PMP20_ASPFU,"[pmp20, AFUA_6G02280]",330879,[O43099],Putative peroxiredoxin pmp20,MSGLKAGDSFPSDVVFSYIPWSEDKGEITACGIPINYNASKEWADK...
1,P15494,True,Swiss-Prot,None,BEV1A_BETPN,"[BETVIA, BETVI]",3505,[Q96369],Major pollen allergen Bet v 1-A,MGVFNYETETTSVIPAARLFKAFILDGDNLFPKVAPQAISSVENIE...
2,Q9T0M8,True,Swiss-Prot,None,PROF2_PARJU,[PRO2],33127,[],Profilin-2,MSWQAYVDDHLMCDVGDGNTLASAAIIGHDGSVWAQSANFPQLKPE...
3,Q39967,True,Swiss-Prot,None,ALL5_HEVBR,[],3981,[],Major latex allergen Hev b 5,MASVEVESAATALPKNETPEVTKAEETKTEEPAAPPASEQETADAT...
4,P45431,True,Swiss-Prot,None,BEV1B_BETPN,[BETV1B],3505,[],Major pollen allergen Bet v 1-B,MGVFNYETETTSVIPAARLFKAFILEGDTLIPKVAPQAISSVENIE...
5,Q43483,True,Swiss-Prot,None,IAA1_HORVU,[IAM1],4513,[P16968],Alpha-amylase inhibitor BMAI-1,PTSVAVDQGSMVSNSPGEWCWPGMGYPVYPFPRCRALVKSQCAGGQ...
6,A4GDU5,True,Swiss-Prot,None,PROBM_OLEEU,[],4146,[],Profilin-2,MSWQTYVDDHLMCDIEGHEGHRLTAAAIVGQDGSVWAQSATFPQFK...
7,A4GDU3,True,Swiss-Prot,None,PROBL_OLEEU,[],4146,[],Profilin-4,MSWQSYVDDHLMCDIEGHEGHRLTAAAIVGHDGSVWAQSATFPQFK...
8,A4GDU2,True,Swiss-Prot,None,PROBK_OLEEU,[],4146,[],Profilin-3,MSWQTYVDDHLMCDIEGHEGHRLTAAAIVGHDGSVWAQSATFPQFK...
9,A4GDU0,True,Swiss-Prot,None,PROBJ_OLEEU,[],4146,[],Profilin-2,MSWQTYVDDHLMCDIEGHEGHRLTAAAIVGHDGSVWAQSATFPQFK...


In [6]:
uniprot_non_allergen_df.head(1)

,acc_num,allergen,db,ec_num,entry_name,gene_names,organism_name,other_acc_nums,protein_name,sequence
0,C6DJ78,False,Swiss-Prot,None,TTCA_PECCP,"[ttcA, PC1_2316]",561230,[],tRNA 2-thiocytidine biosynthesis protein TtcA,MSENQQINQKQQYNLNKLQKRLRRNVGEAIADFNMIEEGDRIMVCL...


In [8]:
print uniprot_allergen_df.shape, uniprot_non_allergen_df.shape

(601, 10) (464738, 10)


In [12]:
# with open('pickles/uniprot_allergen_df.pkl', 'w') as fout:
#     pickle.dump(uniprot_allergen_df, fout)
# with open('pickles/uniprot_non_allergen_df.pkl', 'w') as fout:
#     pickle.dump(uniprot_non_allergen_df, fout)

In [6]:
with open('pickles/uniprot_allergen_df.pkl', 'r') as fin:
    uniprot_allergen_df = pickle.load(fin)
with open('pickles/uniprot_non_allergen_df.pkl', 'r') as fin:
    uniprot_non_allergen_df = pickle.load(fin)

uniprot_allergen_df = uniprot_allergen_df['sequence'].to_frame()
uniprot_allergen_df['allergen'] = 1
uniprot_allergen_df = add_protein_characteristics(uniprot_allergen_df)

uniprot_non_allergen_df = uniprot_non_allergen_df['sequence'].to_frame()
uniprot_non_allergen_df['allergen'] = 0
uniprot_non_allergen_df = add_protein_characteristics(uniprot_non_allergen_df)

# with open('pickles/uniprot_allergen_df_with_features.pkl', 'w') as fout:
#     pickle.dump(uniprot_allergen_df, fout)
# with open('pickles/uniprot_non_allergen_df_with_features.pkl', 'w') as fout:
#     pickle.dump(uniprot_non_allergen_df, fout)

# Allergen Online

In [21]:
fasta_columns = ['db', 'identifier', 'entry_name', 'protein_name', 'organism_name', 'gene_name', 'sequence']
parsed_fasta = parse_fasta_str(open('data/allergen_online_allergens.fasta').read())
ao_allergen_df = pd.DataFrame(data=parsed_fasta, columns=fasta_columns)
ao_allergen_df = ao_allergen_df.drop_duplicates(subset='sequence')

# ao_identifiers = ao_allergen_df['identifier'].tolist()
# for i in range(0, len(ao_identifiers), 200):
#     if i == 0:
#         ao_allergen_df = convert_acc_nums_to_df(str(ao_identifiers[i:i+200]))
#     else:
#         ao_allergen_df = ao_allergen_df.append(convert_acc_nums_to_df(str(ao_identifiers[i:i+200])))

In [22]:
ao_allergen_df.head(1)

,db,identifier,entry_name,protein_name,organism_name,gene_name,sequence
0,tr,B0KZJ6,B0KZJ6_ACASI,Allergen Aca s 13,Acarus siro,None,MVQINGSYKLEKSDNFDAFLKELGLNFVTRNLAKSATPTVEVSVNG...


In [24]:
# with open('pickles/ao_allergen_df.pkl', 'w') as fout:
#     pickle.dump(ao_df, fout)

In [7]:
with open('pickles/ao_allergen_df.pkl', 'r') as fin:
    ao_allergen_df = pickle.load(fin)

ao_allergen_df = uniprot_allergen_df['sequence'].to_frame()
ao_allergen_df['allergen'] = 1
ao_allergen_df = add_protein_characteristics(ao_allergen_df)

# with open('pickles/ao_allergen_df_with_features.pkl', 'w') as fout:
#     pickle.dump(ao_allergen_df, fout)

In [22]:
add_protein_characteristics(ao_allergen_df).shape

(601, 28)

# AllerHunter

In [23]:
base = 'data/allerHunterData/'
paths = ['testingdata/testing.allergen.fa', 'testingdata/testing.homolog.fa', 'testingdata/testing.putative_non_allergen.fa',
        'trainingdata/training.allergen.fa', 'trainingdata/training.homolog.fa', 'trainingdata/training.putative_non_allergen.fa',
        'independentdata/indp.allergen.fa', 'independentdata/indp.homolog.fa', 'independentdata/indp.putative_non_allergen.fa']
seqs = []

for path in paths:
    with open(base + path) as fin:
        seqs.append(get_sequences_from_fasta(fin))

ah_test_allergens, ah_test_APNs, ah_test_DPNs = seqs[0:3]
ah_train_allergens, ah_train_APNs, ah_train_DPNs = seqs[3:6]
ah_ind_allergens, ah_ind_APNs, ah_ind_DPNs = seqs[6:9]

ah_allergen_df = pd.DataFrame({'sequence': ah_test_allergens + ah_train_allergens + ah_ind_allergens})
ah_non_allergen_df = pd.DataFrame({'sequence': ah_test_APNs + ah_test_DPNs + ah_train_APNs + ah_train_DPNs + ah_ind_APNs + ah_ind_DPNs})

ah_allergen_df['allergen'] = 1
ah_non_allergen_df['allergen'] = 0

with open('pickles/ah_allergen_df.pkl', 'w') as fout:
    pickle.dump(ah_allergen_df, fout)
with open('pickles/ah_non_allergen_df.pkl', 'w') as fout:
    pickle.dump(ah_non_allergen_df, fout)

ah_allergen_df = add_protein_characteristics(ah_allergen_df)
ah_non_allergen_df = add_protein_characteristics(ah_non_allergen_df)

with open('pickles/ah_allergen_df_with_features.pkl', 'w') as fout:
    pickle.dump(ah_allergen_df, fout)
with open('pickles/ah_non_allergen_df_with_features.pkl', 'w') as fout:
    pickle.dump(ah_non_allergen_df, fout)
    
# base = 'pickles/'

# paths = ['ah_allergen_df_with_features.pkl', 'ah_non_allergen_df_with_features.pkl', 'ah_ind_allergen_df_with_features.pkl',
#         'ah_ind_non_allergen_df_with_features.pkl']


# paths = ['ah_allergen_df_with_features.pkl', 'ah_non_allergen_df_with_features.pkl', 'ah_ind_allergen_df_with_features.pkl',
#         'ah_ind_non_allergen_df_with_features.pkl']
# dataframes = [ah_allergen_df, ah_non_allergen_df, ah_ind_allergen_df, ah_ind_non_allergen_df]
# for p,d in zip(paths, dataframes):
#     d = add_protein_characteristics(d)
#     d = filter_df_by_seq_len(d)
#     with open(base + p, 'w') as fout:
#         pickle.dump(d, fout)

# Final ABT

In [3]:
base = 'pickles/'
paths = ['ah_allergen_df_with_features.pkl', 'ah_non_allergen_df_with_features.pkl', 'uniprot_allergen_df_with_features.pkl',
        'uniprot_non_allergen_df_with_features.pkl', 'ao_allergen_df_with_features.pkl']
dataframes = []

for path in paths:
    with open(base + path) as fin:
        dataframes.append(pickle.load(fin))
ah_allergen_df, ah_non_allergen_df, uniprot_allergen_df, uniprot_non_allergen_df, ao_allergen_df = dataframes

In [4]:
for d in dataframes: 
    print d.shape

(1405, 28)
(12474, 28)
(601, 28)
(464738, 28)
(601, 28)


In [5]:
allergen_df = ah_allergen_df.append(uniprot_allergen_df, ignore_index=True).drop_duplicates(subset='sequence')\
                            .append(ao_allergen_df, ignore_index=True).drop_duplicates(subset='sequence')
    
non_allergen_df = ah_non_allergen_df.append(uniprot_non_allergen_df, ignore_index=True).drop_duplicates(subset='sequence')

df = allergen_df.append(non_allergen_df, ignore_index=True).drop_duplicates(subset='sequence')

In [6]:
print allergen_df.shape, non_allergen_df.shape, df.shape, df[df['allergen'] == 1].shape, df[df['allergen'] == 0].shape

(1624, 28) (466674, 28) (468095, 28) (1624, 28) (466471, 28)


In [7]:
df = filter_df_by_sim_among_positives(df, 0.90)

In [8]:
print df.shape, df[df['allergen'] == 1].shape, df[df['allergen'] == 0].shape

(467470, 28) (999, 28) (466471, 28)


In [9]:
df = filter_df_by_seq_len(df, lengths=(20, 700))

In [10]:
print df.shape, df[df['allergen'] == 1].shape, df[df['allergen'] == 0].shape

(419602, 28) (914, 28) (418688, 28)


In [11]:
with open('pickles/allergen_df_with_features.pkl', 'w') as fout:
    pickle.dump(allergen_df, fout)
with open('pickles/non_allergen_df_with_features.pkl', 'w') as fout:
    pickle.dump(non_allergen_df, fout)
with open('pickles/full_ABT_df.pkl', 'w') as fout:
    pickle.dump(df, fout)